In [1]:
%%capture
%load_ext sql
%sql sqlite:///../panopticon-working.db

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect("../panopticon-working.db")
cur = conn.cursor()

In [4]:
def query_to_df(query):
    rows = cur.execute(query).fetchall()
    cols = [c[0] for c in cur.description]
    return pd.DataFrame(rows, columns=cols)

In [5]:
query_to_df("""
SELECT
    id,
    datetime(started / 1000, "unixepoch") as started
FROM game
ORDER BY started DESC
LIMIT 5
""")

,id,started
0,-MXYQ9a5RSTLHS0UQJBz,2021-04-05 19:05:46
1,-MWlDBQYhwmL669PLmOE,2021-03-27 01:11:43
2,-MVhrjj9nlUeBmRWBav2,2021-03-13 23:15:07
3,-MVh4GQ9QemRmxVbshn2,2021-03-13 19:34:31
4,-MVPBzwRt28ARpdirFcf,2021-03-10 03:36:32


In [6]:
%%sql
SELECT
    (SELECT COUNT(1) FROM game) AS n_games,
    (SELECT COUNT(1) FROM valid_game) AS n_valid_games,
    (SELECT COUNT(1) FROM completed_game) AS n_valid_games_w_decision
FROM game
LIMIT 1;

 * sqlite:///../panopticon-working.db
Done.


n_games,n_valid_games,n_valid_games_w_decision
405,233,181


In [7]:
%%sql
DROP VIEW IF EXISTS decisions_per_game;
CREATE VIEW decisions_per_game AS
SELECT
    game.id AS game_id,
    COUNT(DISTINCT decision.id) AS n_decisions
FROM valid_game AS game
    LEFT JOIN decision
    ON game.id == decision.game
GROUP BY game_id;

 * sqlite:///../panopticon-working.db
Done.
Done.


[]

In [8]:
%%sql
SELECT
    n_decisions,
    COUNT(1) AS n_games
FROM decisions_per_game
GROUP BY n_decisions
ORDER BY n_decisions ASC;

 * sqlite:///../panopticon-working.db
Done.


n_decisions,n_games
0,52
1,124
2,29
3,16
4,8
5,1
10,1
15,1
32,1


In [9]:
%%sql
SELECT
    n_decisions,
    rationale,
    recommendations,
    suspect,
    submitted
FROM decision
    LEFT JOIN decisions_per_game
    ON decision.game == decisions_per_game.game_id
WHERE decisions_per_game.n_decisions > 4

 * sqlite:///../panopticon-working.db
Done.


n_decisions,rationale,recommendations,suspect,submitted
10,E e e e e e e e ee. E e ee e e e e e e e e e e e,H h h h h hh h h h h hch h h h,Shopper #6871,1595964024528
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964689771
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964698476
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964701208
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964701369
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964702005
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964702635
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964703309
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964703932
10,closer to funny number\noh he also committed a hate crime,use less funny numbers use less funny numbers use less funny numbers use less funny numbersuse less funny numbers,Shopper #6871,1595964704726


In [10]:
%%sql
SELECT
    game.*,
    n_decisions
FROM valid_game AS game
    LEFT JOIN decisions_per_game
    ON game.id == decisions_per_game.game_id
ORDER BY started ASC
LIMIT 20

 * sqlite:///../panopticon-working.db
Done.


id,hostname,gateway,party,started,deactivated_attention,deactivated_movement,deactivated_risk,n_decisions
-MAYd-P-RDJYOwE3lbwt,panopticonsecurity.herokuapp.com,cdf,None,1592957935518.0,1,1,1,1
-MAhw58jAQEjwkjxMKDP,panopticonsecurity.herokuapp.com,cdf,None,1593130520392.0,0,0,0,2
-MAhw8MsIV8VaW4A4Bmt,panopticonsecurity.herokuapp.com,cdf,None,1593130577371.0,1,1,1,1
-MAn8RejJgJ55NYSYYH9,panopticonsecurity.herokuapp.com,cdf,None,1593217548051.0,0,0,0,1
-MAn7f_cwlFtiFxcHRrl,panopticonsecurity.herokuapp.com,cdf,None,1593217587203.0,1,1,1,1
-MB1bWFk-GJXOS5WgabE,panopticonsecurity.herokuapp.com,cdf,None,1593477263491.0,1,1,1,1
-MB1ci3XH10pd0-XP_ao,panopticonsecurity.herokuapp.com,cdf,None,1593477302384.0,0,0,0,1
-MCKu8W2Z7QR8tzHG5bN,panopticonsecurity.herokuapp.com,ab,quest,1594874183144.0,0,0,0,1
-MDJ7Kx_gZiZ4Eo1DJJI,panopticonsecurity.herokuapp.com,esi,None,1595919537337.0,0,0,0,0
-MDJA8z6u0YHwNbiI6lV,panopticonsecurity.herokuapp.com,esi,None,1595919732050.0,0,0,0,1


In [11]:
%%sql
SELECT DISTINCT hostname FROM game

 * sqlite:///../panopticon-working.db
Done.


hostname
None
localhost
panopticonsecurity.herokuapp.com
